In [9]:
from osgeo import gdal

# Define the input and output file paths
input_file = r'C:\Users\USKD712889\OneDrive - WSP O365\Documents\Projects\Monroe County\Duck Key\Delft3D\NAD832007ft\nad832007ft\Flow\Bathy\Bathy.tif'
output_file = r'C:\Users\USKD712889\OneDrive - WSP O365\Documents\Projects\Monroe County\Duck Key\Delft3D\NAD832007ft\nad832007ft\Flow\Bathy\Bathy.xyz'

# Open the input GeoTIFF file
dataset = gdal.Open(input_file)

# Use GDAL's translate function to convert to XYZ format
gdal.Translate(output_file, dataset, format='XYZ')

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x00000194E343ADC0> >

In [10]:
#removes all 0 z values
input_file = r'C:\Users\USKD712889\OneDrive - WSP O365\Documents\Projects\Monroe County\Duck Key\Delft3D\NAD832007ft\nad832007ft\Flow\Bathy\Bathy.xyz'
output_file = r'C:\Users\USKD712889\OneDrive - WSP O365\Documents\Projects\Monroe County\Duck Key\Delft3D\NAD832007ft\nad832007ft\Flow\Bathy\nozeros_Bathy.xyz'

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        values = line.split()  # Split the line into x, y, z
        if len(values) == 3:  # Ensure there are exactly three values
            x, y, z = map(float, values)  # Convert values to floats
            if z != 0.0:  # Check if z is not 0
                outfile.write(line)  # Write the line to the output file

print("Processing complete.")

Processing complete.


In [12]:
#downsampling
import numpy as np

input_file = r'C:\Users\USKD712889\OneDrive - WSP O365\Documents\Projects\Monroe County\Duck Key\Delft3D\NAD832007ft\nad832007ft\Flow\Bathy\nozeros_Bathy.xyz'
output_file = r'C:\Users\USKD712889\OneDrive - WSP O365\Documents\Projects\Monroe County\Duck Key\Delft3D\NAD832007ft\nad832007ft\Flow\Bathy\reduced_Bathy_Rev0.xyz'

downsample_ratio = 700
precision = 4
buffer_size = 1000

buffer = []

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for idx, line in enumerate(infile):
        if idx % downsample_ratio == 0:  # Downsample
            data = np.array(line.split(), dtype=float)
            data = np.round(data, precision)  # Reduce precision
            buffer.append(f'{data[0]:.{precision}f} {data[1]:.{precision}f} {data[2]:.{precision}f}\n')

        # Write buffer to file when it reaches buffer_size
        if len(buffer) >= buffer_size:
            outfile.writelines(buffer)
            buffer = []  # Clear the buffer

    # Write any remaining lines in the buffer
    if buffer:
        outfile.writelines(buffer)

print("Processing complete.")

Processing complete.


In [1]:
#reverses z values. Delft3D FM requires negative z values. Wave requires positive z values for depth. 
import os

# Function to process a line in the .xyz file
def process_line(line):
    parts = line.split()
    # If the line contains exactly 3 parts (x, y, z), modify the z-coordinate
    if len(parts) == 3:
        parts[2] = str(float(parts[2]) * -1)  # Multiply the z-coordinate by -1
    return " ".join(parts)

# Function to read the input .xyz file, process it, and save to a new file in the same folder
def multiply_z_by_negative_one(input_file):
    # Get the directory of the input file
    input_dir = os.path.dirname(input_file)
    
    # Get the file name without the extension and append '_z_modified' to it
    input_filename = os.path.splitext(os.path.basename(input_file))[0]
    output_file = os.path.join(input_dir, f"{input_filename}_z_modified.xyz")
    
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            outfile.write(process_line(line) + '\n')
    
    print(f"All z-values in {input_file} have been multiplied by -1 and saved to {output_file}.")

# Example usage
input_file = r'C:\Users\USKD712889\OneDrive - WSP O365\Documents\Projects\Monroe County\Duck Key\Delft3D\NAD832007ft\nad832007ft\Flow\Bathy\reduced_bathy.xyz'  # Replace with your input .xyz file
multiply_z_by_negative_one(input_file)


All z-values in C:\Users\USKD712889\OneDrive - WSP O365\Documents\Projects\Monroe County\Duck Key\Delft3D\NAD832007ft\nad832007ft\Flow\Bathy\reduced_bathy.xyz have been multiplied by -1 and saved to C:\Users\USKD712889\OneDrive - WSP O365\Documents\Projects\Monroe County\Duck Key\Delft3D\NAD832007ft\nad832007ft\Flow\Bathy\reduced_bathy_z_modified.xyz.


In [1]:
#ft to meters
import os
# Function to process a line in the .xyz file
def process_line(line):
    parts = line.split()
    # If the line contains exactly 3 parts (x, y, z), modify the z-coordinate
    if len(parts) == 3:
        parts[2] = str(float(parts[2]) * 3.28084)  # Multiply the z-coordinate by 3.28084 
    return " ".join(parts)

# Function to read the input .xyz file, process it, and save to a new file in the same folder
def feet_to_meters(input_file):
    # Get the directory of the input file
    input_dir = os.path.dirname(input_file)
    
    # Get the file name without the extension and append '_meters' to it
    input_filename = os.path.splitext(os.path.basename(input_file))[0]
    output_file = os.path.join(input_dir, f"{input_filename}_meters.xyz")
    
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            outfile.write(process_line(line) + '\n')
    
    print(f"All z-values in {input_file} have been turned to meters and saved to {output_file}.")

# Example usage
input_file = r'C:\\Users\\USKD712889\\OneDrive - WSP O365\\Documents\\Projects\\Monroe County\\Duck Key\\Delft3D\\NAD832007ft\\nad832007ft\\reduced_Bathy.xyz'  # Replace with your input .xyz file
feet_to_meters(input_file)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\\\Users\\\\USKD712889\\\\OneDrive - WSP O365\\\\Documents\\\\Projects\\\\Monroe County\\\\Duck Key\\\\Delft3D\\\\NAD832007ft\\\\nad832007ft\\\\reduced_Bathy.xyz'